# The Nature Conservancy Fisheries Monitoring

## Data pre-processing

In [1]:
from __future__ import division,print_function
import math, os, sys, re
import PIL
import ujson as json
import numpy as np
from matplotlib import pyplot as plt

import pandas as pd
import bcolz

from IPython.lib.display import FileLink

import theano
import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Input, Embedding, Reshape, merge, LSTM, Bidirectional
from keras.layers import TimeDistributed, Activation, SimpleRNN, GRU
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers.pooling import GlobalAveragePooling2D
from keras.regularizers import l2, activity_l2, l1, activity_l1
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD, RMSprop, Adam
from keras.utils.layer_utils import layer_from_config
from keras.metrics import categorical_crossentropy, categorical_accuracy
from keras.layers.convolutional import *
from keras.preprocessing import image, sequence
from keras.preprocessing.text import Tokenizer
from keras.utils.layer_utils import convert_all_kernels_in_model
from keras.applications.resnet50 import identity_block, conv_block
from keras.applications.inception_v3 import conv2d_bn

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


In [2]:
def get_batches(dirname, gen = image.ImageDataGenerator(), shuffle = True, batch_size = 64, class_mode = 'categorical',
                target_size = (224,224)):
    return gen.flow_from_directory(dirname, target_size = target_size, class_mode = class_mode, shuffle = shuffle, 
                                   batch_size = batch_size)

In [3]:
def get_classes(path):
    trn_batches = get_batches(path+'train', shuffle = False, batch_size = 1)
    val_batches = get_batches(path+'valid', shuffle = False, batch_size = 1)
    test_batches = get_batches(path+'test', shuffle = False, batch_size = 1)
    test2_batches = get_batches(path+'test_stg2', shuffle = False, batch_size = 1)
    return (trn_batches.classes, val_batches.classes, to_categorical(trn_batches.classes), 
            to_categorical(val_batches.classes), trn_batches.filenames, val_batches.filenames, 
            test_batches.filenames, test2_batches.filenames)

In [4]:
path = '/home/ubuntu/courses/deeplearning1/data/fish/'

In [5]:
batch_size = 32

In [6]:
trn_batches = get_batches(path+'train', batch_size = batch_size, shuffle = False, target_size = (360, 640))
val_batches = get_batches(path+'valid', batch_size = batch_size, shuffle = False, target_size = (360, 640))
test_batches = get_batches(path+'test', batch_size = batch_size, shuffle = False, target_size = (360, 640))
test2_batches = get_batches(path+'test_stg2', batch_size = batch_size, shuffle = False, target_size = (360, 640))

Found 3277 images belonging to 8 classes.
Found 500 images belonging to 8 classes.
Found 1000 images belonging to 1 classes.
Found 12153 images belonging to 1 classes.


In [7]:
#training data augmentation
#gen_aug = image.ImageDataGenerator(rotation_range = 15, height_shift_range = 0.05, shear_range = 0.1, 
#                                  channel_shift_range = 20, width_shift_range = 0.1)
#trn_aug_batches = get_batches(path+'train', gen = gen_aug, batch_size = batch_size, shuffle = False, 
#                              target_size = (360, 640))

In [7]:
(trn_classes, val_classes, trn_labels, val_labels, trn_filenames, val_filenames, 
 test_filenames, test2_filenames) = get_classes(path)

Found 3277 images belonging to 8 classes.
Found 500 images belonging to 8 classes.
Found 1000 images belonging to 1 classes.
Found 12153 images belonging to 1 classes.


In [8]:
trn_raw_filenames = [f.split('/')[-1] for f in trn_filenames]
val_raw_filenames = [f.split('/')[-1] for f in val_filenames]
test_raw_filenames = [f.split('/')[-1] for f in test_filenames]
test2_raw_filenames = [f.split('/')[-1] for f in test2_filenames]

In [9]:
#bcolz speeds up saving and loading np.array process
def save_array(fname, arr):
    c = bcolz.carray(arr, rootdir = fname, mode = 'w')
    c.flush()

In [10]:
def load_array(fname):
    return bcolz.open(fname)[:]

## VGG model

### Calculate convolutional features based on pre-trained model

In [11]:
vgg_mean = np.array([123.68, 116.779, 103.939], dtype = np.float32).reshape((3,1,1))
def vgg_preprocess(x):
    x = x - vgg_mean
    return x[:, ::-1] # reverse axis rgb->bgr

In [12]:
#The VGG 16 Imagenet model with batch normalization for the dense layers
class Vgg16BN():

    def __init__(self, size = (224,224), include_top = True):
        self.FILE_PATH = 'http://www.platform.ai/models/'
        self.create(size, include_top)

    def ConvBlock(self, layers, filters):
        model = self.model
        for i in range(layers):
            model.add(ZeroPadding2D((1, 1)))
            model.add(Convolution2D(filters, 3, 3, activation='relu'))
        model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    def FCBlock(self):
        model = self.model
        model.add(Dense(4096, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))

    def create(self, size, include_top):
        if size != (224,224):
            include_top = False

        model = self.model = Sequential()
        model.add(Lambda(vgg_preprocess, input_shape = (3,) + size, output_shape = (3,) + size))

        self.ConvBlock(2, 64)
        self.ConvBlock(2, 128)
        self.ConvBlock(3, 256)
        self.ConvBlock(3, 512)
        self.ConvBlock(3, 512)

        if not include_top:
            fname = 'vgg16_bn_conv.h5'
            model.load_weights(get_file(fname, self.FILE_PATH+fname, cache_subdir='models'))
            return

        model.add(Flatten())
        self.FCBlock()
        self.FCBlock()
        model.add(Dense(1000, activation='softmax'))
        
        fname = 'vgg16_bn.h5'
        model.load_weights(get_file(fname, self.FILE_PATH+fname, cache_subdir='models'))

In [13]:
vgg = Vgg16BN((360, 640)).model

In [14]:
vgg.pop()  #delete the last MaxPooling layer

In [15]:
vgg.compile(optimizer = Adam(), loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [16]:
vgg.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 360, 640)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 362, 642)   0           lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 360, 640)  1792        zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
zeropadding2d_2 (ZeroPadding2D)  (None, 64, 362, 642)  0           convolution2d_1[0][0]            
___________________________________________________________________________________________

In [ ]:
conv_trn_feat = vgg.predict_generator(trn_batches, trn_batches.nb_sample)

In [ ]:
conv_val_feat = vgg.predict_generator(val_batches, val_batches.nb_sample)

In [ ]:
conv_test_feat = vgg.predict_generator(test_batches, test_batches.nb_sample)

In [22]:
conv_test2_feat = vgg.predict_generator(test2_batches, test2_batches.nb_sample)

In [ ]:
save_array(path+'results/conv_trn_feat.dat', conv_trn_feat)

In [ ]:
save_array(path+'results/conv_val_feat.dat', conv_val_feat)

In [ ]:
save_array(path+'results/conv_test_feat.dat', conv_test_feat)

In [25]:
save_array(path+'results/conv_test2_feat.dat', conv_test2_feat)

In [17]:
conv_trn_feat = load_array(path+'results/conv_trn_feat.dat')

In [18]:
conv_val_feat = load_array(path+'results/conv_val_feat.dat')

In [19]:
conv_test_feat = load_array(path+'results/conv_test_feat.dat')

In [20]:
conv_test2_feat = load_array(path+'results/conv_test2_feat.dat')

In [ ]:
#training data augmentation
#conv_trn_aug_feat = vgg.predict_generator(trn_aug_batches, trn_aug_batches.nb_sample * 5)
#save_array(path+'results/conv_trn_aug_feat.dat', conv_trn_aug_feat)
#conv_trn_aug_feat = load_array(path+'results/conv_trn_aug_feat.dat')

In [ ]:
#training data augmentation
#conv_trn_feat = np.concatenate([conv_trn_feat, conv_trn_aug_feat[:3277*3, :, :, :]])
#trn_labels = np.concatenate([trn_labels] * 4)
#del conv_trn_aug_feat

In [21]:
conv_trn_feat.shape, conv_val_feat.shape

((3277, 512, 22, 40), (500, 512, 22, 40))

In [22]:
conv_test_feat.shape, conv_test2_feat.shape

((1000, 512, 22, 40), (12153, 512, 22, 40))

### Train on fully-convolutional network

In [23]:
vgg.layers[-1].output_shape[1:]

(512, 22, 40)

In [24]:
model = Sequential([
        BatchNormalization(axis = 1, input_shape = vgg.layers[-1].output_shape[1:]),
        Convolution2D(128, 3, 3, activation = 'relu', border_mode = 'same'),
        BatchNormalization(axis = 1),
        MaxPooling2D(),
        Convolution2D(128, 3, 3, activation = 'relu', border_mode = 'same'),
        BatchNormalization(axis = 1),
        MaxPooling2D(),
        Convolution2D(128, 3, 3, activation = 'relu', border_mode = 'same'),
        BatchNormalization(axis = 1),
        MaxPooling2D((1, 2)),
        Convolution2D(8, 3, 3, border_mode = 'same'),
        GlobalAveragePooling2D(),
        Activation('softmax')
    ])

th


In [25]:
model.compile(optimizer = Adam(lr=1e-3), loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [26]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
batchnormalization_1 (BatchNormal(None, 512, 22, 40)   1024        batchnormalization_input_1[0][0] 
____________________________________________________________________________________________________
convolution2d_14 (Convolution2D) (None, 128, 22, 40)   589952      batchnormalization_1[0][0]       
____________________________________________________________________________________________________
batchnormalization_2 (BatchNormal(None, 128, 22, 40)   256         convolution2d_14[0][0]           
____________________________________________________________________________________________________
maxpooling2d_6 (MaxPooling2D)    (None, 128, 11, 20)   0           batchnormalization_2[0][0]       
___________________________________________________________________________________________

In [27]:
batch_size = 64

In [28]:
model.fit(conv_trn_feat, trn_labels, batch_size = batch_size, nb_epoch = 2, 
          validation_data = (conv_val_feat, val_labels))

Train on 3277 samples, validate on 500 samples
Epoch 1/2
3277/3277 [==============================] - 16s - loss: 0.6890 - acc: 0.7870 - val_loss: 0.9690 - val_acc: 0.7280
Epoch 2/2
3277/3277 [==============================] - 16s - loss: 0.1329 - acc: 0.9628 - val_loss: 0.3531 - val_acc: 0.8940


In [29]:
model.optimizer.lr = 1e-5

In [30]:
model.fit(conv_trn_feat, trn_labels, batch_size = batch_size, nb_epoch = 6, 
          validation_data = (conv_val_feat, val_labels))

Train on 3277 samples, validate on 500 samples
Epoch 1/6
3277/3277 [==============================] - 17s - loss: 0.0394 - acc: 0.9896 - val_loss: 0.1359 - val_acc: 0.9760
Epoch 2/6
3277/3277 [==============================] - 16s - loss: 0.0122 - acc: 0.9985 - val_loss: 0.1046 - val_acc: 0.9780
Epoch 3/6
3277/3277 [==============================] - 16s - loss: 0.0047 - acc: 0.9988 - val_loss: 0.1238 - val_acc: 0.9680
Epoch 4/6
3277/3277 [==============================] - 16s - loss: 0.0024 - acc: 0.9997 - val_loss: 0.0949 - val_acc: 0.9720
Epoch 5/6
3277/3277 [==============================] - 17s - loss: 0.0012 - acc: 1.0000 - val_loss: 0.0937 - val_acc: 0.9760
Epoch 6/6
3277/3277 [==============================] - 17s - loss: 5.9466e-04 - acc: 1.0000 - val_loss: 0.0896 - val_acc: 0.9800


### Conduct model ensemble based on entire training set

In [31]:
conv_ens_feat = np.concatenate((conv_trn_feat, conv_val_feat), axis = 0)

In [32]:
ens_labels = np.concatenate((trn_labels, val_labels), axis = 0)

In [33]:
conv_ens_feat.shape, ens_labels.shape

((3777, 512, 22, 40), (3777, 8))

In [34]:
def fully_conv(i):
    model = Sequential([
        BatchNormalization(axis = 1, input_shape = vgg.layers[-1].output_shape[1:]),
        Convolution2D(128, 3, 3, activation = 'relu', border_mode = 'same'),
        BatchNormalization(axis = 1),
        MaxPooling2D(),
        Convolution2D(128, 3, 3, activation = 'relu', border_mode = 'same'),
        BatchNormalization(axis = 1),
        MaxPooling2D(),
        Convolution2D(128, 3, 3, activation = 'relu', border_mode = 'same'),
        BatchNormalization(axis = 1),
        MaxPooling2D((1, 2)),
        Convolution2D(8, 3, 3, border_mode = 'same'),
        GlobalAveragePooling2D(),
        Activation('softmax')
    ])
    model.compile(optimizer = Adam(lr=1e-3), loss = 'categorical_crossentropy', metrics = ['accuracy'])
    model.fit(conv_ens_feat, ens_labels, batch_size = batch_size, nb_epoch = 2)
    model.optimizer.lr = 1e-5
    model.fit(conv_ens_feat, ens_labels, batch_size = batch_size, nb_epoch = 6)
    model.save_weights(path+'models/vgg_'+str(i)+'.h5')

In [ ]:
for i in range(100):
    print('Training model {}'.format(i))
    fully_conv(i)

Training model 0
th
Epoch 1/2
3777/3777 [==============================] - 17s - loss: 0.5805 - acc: 0.8155    
Epoch 2/2
3777/3777 [==============================] - 17s - loss: 0.1415 - acc: 0.9603    
Epoch 1/6
1984/3777 [==============>...............] - ETA: 8s - loss: 0.0904 - acc: 0.9803

In [43]:
#Predict test data from test_stg1
preds_ens = []
for i in range(100):
    model.load_weights(path+'models/vgg_'+str(i)+'.h5')
    preds_ens.append(model.predict(conv_test_feat, batch_size = batch_size))
    if i % 10 == 9:
        print('Finish predicting {} samples'.format(i+1))

Finish predicting 10 samples
Finish predicting 20 samples
Finish predicting 30 samples
Finish predicting 40 samples
Finish predicting 50 samples
Finish predicting 60 samples
Finish predicting 70 samples
Finish predicting 80 samples
Finish predicting 90 samples
Finish predicting 100 samples


In [38]:
#Predict test data from test_stg2
preds2_ens = []
for i in range(100):
    model.load_weights(path+'models/vgg_all_'+str(i)+'.h5')
    preds2_ens.append(model.predict(conv_test2_feat, batch_size = batch_size))
    if i % 10 == 9:
        print('Finish predicting {} samples'.format(i+1))

Finish predicting 10 samples
Finish predicting 20 samples
Finish predicting 30 samples
Finish predicting 40 samples
Finish predicting 50 samples
Finish predicting 60 samples
Finish predicting 70 samples
Finish predicting 80 samples
Finish predicting 90 samples
Finish predicting 100 samples


In [44]:
preds_ens_vgg = np.stack(preds_ens)
preds_ens_vgg.shape

(100, 1000, 8)

In [45]:
save_array(path+'results/preds_ens_vgg.dat', preds_ens_vgg)

In [39]:
preds2_ens_vgg = np.stack(preds2_ens)
preds2_ens_vgg.shape

(100, 12153, 8)

In [40]:
save_array(path+'results/preds2_ens_vgg.dat', preds2_ens_vgg)

## ResNet model

### Calculate convolutional features based on pre-trained model

In [35]:
# The Resnet 50 Imagenet model
class Resnet50():

    def __init__(self, size=(224,224), include_top=True):
        self.FILE_PATH = 'http://www.platform.ai/models/'
        self.vgg_mean = np.array([123.68, 116.779, 103.939]).reshape((3,1,1))
        self.create(size, include_top)

    def vgg_preprocess(self, x):
        x = x - self.vgg_mean
        return x[:, ::-1] # reverse axis bgr->rgb

    def create(self, size, include_top):
        input_shape = (3,)+size
        img_input = Input(shape=input_shape)
        bn_axis = 1

        x = Lambda(self.vgg_preprocess)(img_input)
        x = ZeroPadding2D((3, 3))(x)
        x = Convolution2D(64, 7, 7, subsample=(2, 2), name='conv1')(x)
        x = BatchNormalization(axis=bn_axis, name='bn_conv1')(x)
        x = Activation('relu')(x)
        x = MaxPooling2D((3, 3), strides=(2, 2))(x)

        x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
        x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
        x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')

        x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
        for n in ['b','c','d']: x = identity_block(x, 3, [128, 128, 512], stage=3, block=n)
        x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a')
        for n in ['b','c','d', 'e', 'f']: x = identity_block(x, 3, [256, 256, 1024], stage=4, block=n)

        x = conv_block(x, 3, [512, 512, 2048], stage=5, block='a')
        x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b')
        x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c')

        if include_top:
            x = AveragePooling2D((7, 7), name='avg_pool')(x)
            x = Flatten()(x)
            x = Dense(1000, activation='softmax', name='fc1000')(x)
            fname = 'resnet50.h5'
        else:
            fname = 'resnet_nt.h5'

        self.img_input = img_input
        self.model = Model(self.img_input, x)
        convert_all_kernels_in_model(self.model)
        self.model.load_weights(get_file(fname, self.FILE_PATH+fname, cache_subdir='models'))

In [36]:
resnet = Resnet50(size = (360, 640), include_top = False).model

In [37]:
resnet.compile(optimizer = Adam(), loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [38]:
resnet.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 3, 360, 640)   0                                            
____________________________________________________________________________________________________
lambda_2 (Lambda)                (None, 3, 360, 640)   0           input_1[0][0]                    
____________________________________________________________________________________________________
zeropadding2d_14 (ZeroPadding2D) (None, 3, 366, 646)   0           lambda_2[0][0]                   
____________________________________________________________________________________________________
conv1 (Convolution2D)            (None, 64, 180, 320)  9472        zeropadding2d_14[0][0]           
___________________________________________________________________________________________

In [ ]:
conv_trn_feat = resnet.predict_generator(trn_batches, trn_batches.nb_sample)

In [ ]:
conv_val_feat = resnet.predict_generator(val_batches, val_batches.nb_sample)

In [ ]:
conv_test_feat = resnet.predict_generator(test_batches, test_batches.nb_sample)

In [52]:
conv_test2_feat = resnet.predict_generator(test2_batches, test2_batches.nb_sample)

In [ ]:
save_array(path+'results/conv_trn_feat_resnet.dat', conv_trn_feat)

In [ ]:
save_array(path+'results/conv_val_feat_resnet.dat', conv_val_feat)

In [ ]:
save_array(path+'results/conv_test_feat_resnet.dat', conv_test_feat)

In [53]:
save_array(path+'results/conv_test2_feat_resnet.dat', conv_test2_feat)

In [39]:
conv_trn_feat = load_array(path+'results/conv_trn_feat_resnet.dat')

In [40]:
conv_val_feat = load_array(path+'results/conv_val_feat_resnet.dat')

In [41]:
conv_test_feat = load_array(path+'results/conv_test_feat_resnet.dat')

In [42]:
conv_test2_feat = load_array(path+'results/conv_test2_feat_resnet.dat')

In [43]:
conv_trn_feat.shape, conv_val_feat.shape

((3277, 2048, 12, 20), (500, 2048, 12, 20))

In [44]:
conv_test_feat.shape, conv_test2_feat.shape

((1000, 2048, 12, 20), (12153, 2048, 12, 20))

### Train on fully-convolutional network (version 1)

In [45]:
resnet.layers[-1].output_shape[1:]

(2048, 12, 20)

In [46]:
model = Sequential([
        BatchNormalization(axis = 1, input_shape = resnet.layers[-1].output_shape[1:]),
        Convolution2D(512, 3, 3, activation = 'relu', border_mode = 'same'),
        BatchNormalization(axis = 1),
        MaxPooling2D(),
        Convolution2D(256, 3, 3, activation = 'relu', border_mode = 'same'),
        BatchNormalization(axis = 1),
        MaxPooling2D(),
        Convolution2D(128, 3, 3, activation = 'relu', border_mode = 'same'),
        BatchNormalization(axis = 1),
        MaxPooling2D((1, 2)),
        Convolution2D(8, 3, 3, border_mode = 'same'),
        GlobalAveragePooling2D(),
        Activation('softmax')
    ])

th


In [47]:
model.compile(optimizer = Adam(lr=1e-3), loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [48]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
batchnormalization_5 (BatchNormal(None, 2048, 12, 20)  4096        batchnormalization_input_2[0][0] 
____________________________________________________________________________________________________
convolution2d_18 (Convolution2D) (None, 512, 12, 20)   9437696     batchnormalization_5[0][0]       
____________________________________________________________________________________________________
batchnormalization_6 (BatchNormal(None, 512, 12, 20)   1024        convolution2d_18[0][0]           
____________________________________________________________________________________________________
maxpooling2d_10 (MaxPooling2D)   (None, 512, 6, 10)    0           batchnormalization_6[0][0]       
___________________________________________________________________________________________

In [49]:
batch_size = 64

In [24]:
model.fit(conv_trn_feat, trn_labels, batch_size = batch_size, nb_epoch = 2, 
          validation_data = (conv_val_feat, val_labels))

Train on 3277 samples, validate on 500 samples
Epoch 1/2
3277/3277 [==============================] - 39s - loss: 0.6184 - acc: 0.8096 - val_loss: 4.9963 - val_acc: 0.4560
Epoch 2/2
3277/3277 [==============================] - 39s - loss: 0.0898 - acc: 0.9780 - val_loss: 0.7236 - val_acc: 0.7740


In [25]:
model.optimizer.lr = 1e-5

In [26]:
model.fit(conv_trn_feat, trn_labels, batch_size = batch_size, nb_epoch = 6, 
          validation_data = (conv_val_feat, val_labels))

Train on 3277 samples, validate on 500 samples
Epoch 1/6
3277/3277 [==============================] - 40s - loss: 0.0342 - acc: 0.9908 - val_loss: 0.1507 - val_acc: 0.9660
Epoch 2/6
3277/3277 [==============================] - 40s - loss: 0.0274 - acc: 0.9933 - val_loss: 0.1266 - val_acc: 0.9660
Epoch 3/6
3277/3277 [==============================] - 40s - loss: 0.0103 - acc: 0.9982 - val_loss: 0.1349 - val_acc: 0.9660
Epoch 4/6
3277/3277 [==============================] - 40s - loss: 0.0048 - acc: 0.9985 - val_loss: 0.1174 - val_acc: 0.9720
Epoch 5/6
3277/3277 [==============================] - 40s - loss: 0.0114 - acc: 0.9973 - val_loss: 0.1417 - val_acc: 0.9720
Epoch 6/6
3277/3277 [==============================] - 40s - loss: 0.0205 - acc: 0.9945 - val_loss: 0.1749 - val_acc: 0.9580


### Train on fully-convolutional network (version 2)

In [50]:
model = Sequential([
        BatchNormalization(axis = 1, input_shape = resnet.layers[-1].output_shape[1:]),
        Convolution2D(256, 3, 3, activation = 'relu', border_mode = 'same'),
        BatchNormalization(axis = 1),
        MaxPooling2D(),
        Convolution2D(256, 3, 3, activation = 'relu', border_mode = 'same'),
        BatchNormalization(axis = 1),
        MaxPooling2D(),
        Convolution2D(256, 3, 3, activation = 'relu', border_mode = 'same'),
        BatchNormalization(axis = 1),
        MaxPooling2D((1, 2)),
        Convolution2D(8, 3, 3, border_mode = 'same'),
        GlobalAveragePooling2D(),
        Activation('softmax')
    ])

th


In [51]:
model.compile(optimizer = Adam(lr=1e-3), loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [52]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
batchnormalization_9 (BatchNormal(None, 2048, 12, 20)  4096        batchnormalization_input_3[0][0] 
____________________________________________________________________________________________________
convolution2d_22 (Convolution2D) (None, 256, 12, 20)   4718848     batchnormalization_9[0][0]       
____________________________________________________________________________________________________
batchnormalization_10 (BatchNorma(None, 256, 12, 20)   512         convolution2d_22[0][0]           
____________________________________________________________________________________________________
maxpooling2d_13 (MaxPooling2D)   (None, 256, 6, 10)    0           batchnormalization_10[0][0]      
___________________________________________________________________________________________

In [53]:
batch_size = 64

In [68]:
model.fit(conv_trn_feat, trn_labels, batch_size = batch_size, nb_epoch = 2, 
          validation_data = (conv_val_feat, val_labels))

Train on 3277 samples, validate on 500 samples
Epoch 1/2
3277/3277 [==============================] - 25s - loss: 0.6222 - acc: 0.8029 - val_loss: 1.8518 - val_acc: 0.6360
Epoch 2/2
3277/3277 [==============================] - 25s - loss: 0.0974 - acc: 0.9744 - val_loss: 1.4139 - val_acc: 0.6600


In [69]:
model.optimizer.lr = 1e-5

In [70]:
model.fit(conv_trn_feat, trn_labels, batch_size = batch_size, nb_epoch = 6, 
          validation_data = (conv_val_feat, val_labels))

Train on 3277 samples, validate on 500 samples
Epoch 1/6
3277/3277 [==============================] - 25s - loss: 0.0227 - acc: 0.9951 - val_loss: 0.1454 - val_acc: 0.9640
Epoch 2/6
3277/3277 [==============================] - 25s - loss: 0.0105 - acc: 0.9973 - val_loss: 0.1105 - val_acc: 0.9720
Epoch 3/6
3277/3277 [==============================] - 25s - loss: 0.0046 - acc: 0.9994 - val_loss: 0.1082 - val_acc: 0.9720
Epoch 4/6
3277/3277 [==============================] - 25s - loss: 0.0047 - acc: 0.9991 - val_loss: 0.0933 - val_acc: 0.9760
Epoch 5/6
3277/3277 [==============================] - 25s - loss: 6.0614e-04 - acc: 1.0000 - val_loss: 0.0891 - val_acc: 0.9740
Epoch 6/6
3277/3277 [==============================] - 25s - loss: 4.1855e-04 - acc: 1.0000 - val_loss: 0.0901 - val_acc: 0.9740


### Train on fully-convolutional network (with bounding-box)

In [54]:
anno_classes = ['alb', 'bet', 'dol', 'lag', 'other', 'shark', 'yft']

In [55]:
#Find bounding-box with the largest area for each picture
bb_json = {}
for c in anno_classes:
    j = json.load(open('{}annos/{}_labels.json'.format(path, c), 'r'))
    for l in j:
        if 'annotations' in l.keys() and len(l['annotations']) > 0:
            bb_json[l['filename'].split('/')[-1]] = sorted(
                l['annotations'], key=lambda x: x['height']*x['width'])[-1]

In [56]:
empty_bbox = {'height': 0., 'width': 0., 'x': 0., 'y': 0.}

In [57]:
#Set bounding-box to [0, 0, 0, 0] to pictures with no bounding-box information
for f in trn_raw_filenames:
    if not f in bb_json.keys(): bb_json[f] = empty_bbox
for f in val_raw_filenames:
    if not f in bb_json.keys(): bb_json[f] = empty_bbox

In [58]:
trn_sizes = [PIL.Image.open(path+'train/'+f).size for f in trn_filenames]
val_sizes = [PIL.Image.open(path+'valid/'+f).size for f in val_filenames]

In [59]:
#Resize bounding-box to fit the image size (240, 360)
bb_params = ['x', 'y', 'width', 'height']
def convert_bb(bb, size):
    bb = [bb[p] for p in bb_params]
    conv_x = (360. / size[0])
    conv_y = (240. / size[1])
    bb[0] = max(bb[0]*conv_x, 0)
    bb[1] = max(bb[1]*conv_y, 0)
    bb[2] = bb[2]*conv_x
    bb[3] = bb[3]*conv_y
    return bb

In [60]:
trn_bbox = np.stack([convert_bb(bb_json[f], s) for f, s in zip(trn_raw_filenames, trn_sizes)]).astype(np.float32)
val_bbox = np.stack([convert_bb(bb_json[f], s) for f, s in zip(val_raw_filenames, val_sizes)]).astype(np.float32)

In [61]:
inp = Input(resnet.layers[-1].output_shape[1:])
x = BatchNormalization(axis = 1)(inp)
x = Convolution2D(512, 3, 3, activation = 'relu', border_mode = 'same')(x)
x = BatchNormalization(axis = 1)(x)
x = MaxPooling2D()(x)
x = Convolution2D(256, 3, 3, activation = 'relu', border_mode = 'same')(x)
x = BatchNormalization(axis = 1)(x)
x = MaxPooling2D()(x)
x = Convolution2D(128, 3, 3, activation = 'relu', border_mode = 'same')(x)
x = BatchNormalization(axis = 1)(x)
x = MaxPooling2D((1, 2))(x)
x_bb = Convolution2D(4, 3, 3, border_mode = 'same')(x)
x_bb = GlobalAveragePooling2D(name = 'bb')(x_bb)
x_class = Convolution2D(8, 3, 3, border_mode = 'same')(x)
x_class = GlobalAveragePooling2D()(x_class)
x_class = Activation('softmax', name = 'class')(x_class)

th
th


In [62]:
model = Model([inp], [x_bb, x_class])

In [63]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 2048, 12, 20)  0                                            
____________________________________________________________________________________________________
batchnormalization_13 (BatchNorma(None, 2048, 12, 20)  4096        input_2[0][0]                    
____________________________________________________________________________________________________
convolution2d_26 (Convolution2D) (None, 512, 12, 20)   9437696     batchnormalization_13[0][0]      
____________________________________________________________________________________________________
batchnormalization_14 (BatchNorma(None, 512, 12, 20)   1024        convolution2d_26[0][0]           
___________________________________________________________________________________________

In [64]:
model.compile(optimizer = Adam(lr=1e-3), loss = ['mse', 'categorical_crossentropy'], metrics = ['accuracy'],
             loss_weights = [1e-3, 1.])

In [65]:
batch_size = 64

In [101]:
model.fit(conv_trn_feat, [trn_bbox, trn_labels], batch_size = batch_size, nb_epoch = 2, 
          validation_data = (conv_val_feat, [val_bbox, val_labels]))

Train on 3277 samples, validate on 500 samples
Epoch 1/2
3277/3277 [==============================] - 39s - loss: 8.2223 - bb_loss: 7510.2812 - class_loss: 0.7120 - bb_acc: 0.6588 - class_acc: 0.7800 - val_loss: 8.2052 - val_bb_loss: 6297.3249 - val_class_loss: 1.9078 - val_bb_acc: 0.6400 - val_class_acc: 0.6300
Epoch 2/2
3277/3277 [==============================] - 39s - loss: 3.6064 - bb_loss: 3426.8732 - class_loss: 0.1796 - bb_acc: 0.6125 - class_acc: 0.9576 - val_loss: 10.1003 - val_bb_loss: 9568.4498 - val_class_loss: 0.5319 - val_bb_acc: 0.6440 - val_class_acc: 0.8380


In [102]:
model.optimizer.lr = 1e-5

In [103]:
model.fit(conv_trn_feat, [trn_bbox, trn_labels], batch_size = batch_size, nb_epoch = 8, 
          validation_data = (conv_val_feat, [val_bbox, val_labels]))

Train on 3277 samples, validate on 500 samples
Epoch 1/8
3277/3277 [==============================] - 39s - loss: 1.6239 - bb_loss: 1557.0451 - class_loss: 0.0669 - bb_acc: 0.5685 - class_acc: 0.9863 - val_loss: 4.2632 - val_bb_loss: 4103.4724 - val_class_loss: 0.1597 - val_bb_acc: 0.6340 - val_class_acc: 0.9640
Epoch 2/8
3277/3277 [==============================] - 39s - loss: 0.9616 - bb_loss: 930.4514 - class_loss: 0.0311 - bb_acc: 0.5917 - class_acc: 0.9960 - val_loss: 1.0669 - val_bb_loss: 893.1700 - val_class_loss: 0.1738 - val_bb_acc: 0.6580 - val_class_acc: 0.9600
Epoch 3/8
3277/3277 [==============================] - 39s - loss: 0.5728 - bb_loss: 555.5393 - class_loss: 0.0172 - bb_acc: 0.7473 - class_acc: 0.9963 - val_loss: 0.6453 - val_bb_loss: 543.9238 - val_class_loss: 0.1014 - val_bb_acc: 0.7960 - val_class_acc: 0.9660
Epoch 4/8
3277/3277 [==============================] - 39s - loss: 0.3726 - bb_loss: 362.3860 - class_loss: 0.0102 - bb_acc: 0.8013 - class_acc: 0.9994 - va

### Conduct model ensemble based on entire training set (version 1)

In [66]:
conv_ens_feat = np.concatenate((conv_trn_feat, conv_val_feat), axis = 0)

In [67]:
ens_labels = np.concatenate((trn_labels, val_labels), axis = 0)

In [68]:
conv_ens_feat.shape, ens_labels.shape

((3777, 2048, 12, 20), (3777, 8))

In [69]:
def fully_conv(i):
    model = Sequential([
        BatchNormalization(axis = 1, input_shape = resnet.layers[-1].output_shape[1:]),
        Convolution2D(512, 3, 3, activation = 'relu', border_mode = 'same'),
        BatchNormalization(axis = 1),
        MaxPooling2D(),
        Convolution2D(256, 3, 3, activation = 'relu', border_mode = 'same'),
        BatchNormalization(axis = 1),
        MaxPooling2D(),
        Convolution2D(128, 3, 3, activation = 'relu', border_mode = 'same'),
        BatchNormalization(axis = 1),
        MaxPooling2D((1, 2)),
        Convolution2D(8, 3, 3, border_mode = 'same'),
        GlobalAveragePooling2D(),
        Activation('softmax')
    ])
    model.compile(optimizer = Adam(lr=1e-3), loss = 'categorical_crossentropy', metrics = ['accuracy'])
    model.fit(conv_ens_feat, ens_labels, batch_size = batch_size, nb_epoch = 2)
    model.optimizer.lr = 1e-5
    model.fit(conv_ens_feat, ens_labels, batch_size = batch_size, nb_epoch = 6)
    model.save_weights(path+'models/resnet_v1_'+str(i)+'.h5')

In [ ]:
for i in range(100):
    print('Training model {}'.format(i))
    fully_conv(i)

Training model 0
th
Epoch 1/2
2304/3777 [=================>............] - ETA: 10s - loss: 0.7970 - acc: 0.7470

In [81]:
preds_ens = []
for i in range(100):
    model.load_weights(path+'models/resnet_v1_'+str(i)+'.h5')
    preds_ens.append(model.predict(conv_test_feat, batch_size = batch_size))
    if i % 10 == 9:
        print('Finish predicting {} samples'.format(i+1))

Finish predicting 10 samples
Finish predicting 20 samples
Finish predicting 30 samples
Finish predicting 40 samples
Finish predicting 50 samples
Finish predicting 60 samples
Finish predicting 70 samples
Finish predicting 80 samples
Finish predicting 90 samples
Finish predicting 100 samples


In [76]:
preds2_ens = []
for i in range(100):
    model.load_weights(path+'models/resnet_v1_'+str(i)+'.h5')
    preds2_ens.append(model.predict(conv_test2_feat, batch_size = batch_size))
    if i % 10 == 9:
        print('Finish predicting {} samples'.format(i+1))

Finish predicting 10 samples
Finish predicting 20 samples
Finish predicting 30 samples
Finish predicting 40 samples
Finish predicting 50 samples
Finish predicting 60 samples
Finish predicting 70 samples
Finish predicting 80 samples
Finish predicting 90 samples
Finish predicting 100 samples


In [67]:
preds_ens_resnet_v1 = np.stack(preds_ens)
preds_ens_resnet_v1.shape

(100, 1000, 8)

In [68]:
save_array(path+'results/preds_ens_resnet_v1.dat', preds_ens_resnet_v1)

In [62]:
preds2_ens_resnet_v1 = np.stack(preds2_ens)
preds2_ens_resnet_v1.shape

(100, 12153, 8)

In [63]:
save_array(path+'results/preds2_ens_resnet_v1.dat', preds2_ens_resnet_v1)

### Conduct model ensemble based on entire training set (version 2)

In [70]:
def fully_conv(i):
    model = Sequential([
        BatchNormalization(axis = 1, input_shape = resnet.layers[-1].output_shape[1:]),
        Convolution2D(256, 3, 3, activation = 'relu', border_mode = 'same'),
        BatchNormalization(axis = 1),
        MaxPooling2D(),
        Convolution2D(256, 3, 3, activation = 'relu', border_mode = 'same'),
        BatchNormalization(axis = 1),
        MaxPooling2D(),
        Convolution2D(256, 3, 3, activation = 'relu', border_mode = 'same'),
        BatchNormalization(axis = 1),
        MaxPooling2D((1, 2)),
        Convolution2D(8, 3, 3, border_mode = 'same'),
        GlobalAveragePooling2D(),
        Activation('softmax')
    ])
    model.compile(optimizer = Adam(lr=1e-3), loss = 'categorical_crossentropy', metrics = ['accuracy'])
    model.fit(conv_ens_feat, ens_labels, batch_size = batch_size, nb_epoch = 2)
    model.optimizer.lr = 1e-5
    model.fit(conv_ens_feat, ens_labels, batch_size = batch_size, nb_epoch = 6)
    model.save_weights(path+'models/resnet_v2_'+str(i)+'.h5')

In [ ]:
for i in range(100):
    print('Training model {}'.format(i))
    fully_conv(i)

In [ ]:
preds_ens = []
for i in range(100):
    model.load_weights(path+'models/resnet_v2_'+str(i)+'.h5')
    preds_ens.append(model.predict(conv_test_feat, batch_size = batch_size))
    if i % 10 == 9:
        print('Finish predicting {} samples'.format(i+1))

In [ ]:
preds2_ens = []
for i in range(100):
    model.load_weights(path+'models/resnet_v2_'+str(i)+'.h5')
    preds2_ens.append(model.predict(conv_test2_feat, batch_size = batch_size))
    if i % 10 == 9:
        print('Finish predicting {} samples'.format(i+1))

In [82]:
preds_ens_resnet_v2 = np.stack(preds_ens)
preds_ens_resnet_v2.shape

(100, 1000, 8)

In [83]:
save_array(path+'results/preds_ens_resnet_v2.dat', preds_ens_resnet_v2)

In [77]:
preds2_ens_resnet_v2 = np.stack(preds2_ens)
preds2_ens_resnet_v2.shape

(100, 12153, 8)

In [78]:
save_array(path+'results/preds2_ens_resnet_v2.dat', preds2_ens_resnet_v2)

### Conduct model ensemble based on entire training set with bounding-box

In [71]:
conv_ens_feat = np.concatenate((conv_trn_feat, conv_val_feat), axis = 0)

In [72]:
ens_labels = np.concatenate((trn_labels, val_labels), axis = 0)

In [73]:
ens_bbox = np.concatenate((trn_bbox, val_bbox), axis = 0)

In [74]:
conv_ens_feat.shape, ens_labels.shape, ens_bbox.shape

((3777, 2048, 12, 20), (3777, 8), (3777, 4))

In [75]:
def fully_conv(i):
    inp = Input(resnet.layers[-1].output_shape[1:])
    x = BatchNormalization(axis = 1)(inp)
    x = Convolution2D(512, 3, 3, activation = 'relu', border_mode = 'same')(x)
    x = BatchNormalization(axis = 1)(x)
    x = MaxPooling2D()(x)
    x = Convolution2D(256, 3, 3, activation = 'relu', border_mode = 'same')(x)
    x = BatchNormalization(axis = 1)(x)
    x = MaxPooling2D()(x)
    x = Convolution2D(128, 3, 3, activation = 'relu', border_mode = 'same')(x)
    x = BatchNormalization(axis = 1)(x)
    x = MaxPooling2D((1, 2))(x)
    x_bb = Convolution2D(4, 3, 3, border_mode = 'same')(x)
    x_bb = GlobalAveragePooling2D(name = 'bb')(x_bb)
    x_class = Convolution2D(8, 3, 3, border_mode = 'same')(x)
    x_class = GlobalAveragePooling2D()(x_class)
    x_class = Activation('softmax', name = 'class')(x_class)
    model = Model([inp], [x_bb, x_class])
    
    model.compile(optimizer = Adam(lr=1e-3), loss = ['mse', 'categorical_crossentropy'], metrics = ['accuracy'],
             loss_weights = [1e-3, 1.])
    model.fit(conv_ens_feat, [ens_bbox, ens_labels], batch_size = batch_size, nb_epoch = 2)
    model.optimizer.lr = 1e-5
    model.fit(conv_ens_feat, [ens_bbox, ens_labels], batch_size = batch_size, nb_epoch = 8)
    model.save_weights(path+'models/resnet_bb_'+str(i)+'.h5')

In [ ]:
for i in range(200):
    print('Training model {}'.format(i))
    fully_conv(i)

Training model 100
th
th
Epoch 1/2
3777/3777 [==============================] - 42s - loss: 7.6784 - bb_loss: 7053.0195 - class_loss: 0.6254 - bb_acc: 0.7098 - class_acc: 0.8226    
Epoch 2/2
3777/3777 [==============================] - 42s - loss: 2.9536 - bb_loss: 2811.7071 - class_loss: 0.1419 - bb_acc: 0.6174 - class_acc: 0.9714    
Epoch 1/8
3777/3777 [==============================] - 42s - loss: 1.6175 - bb_loss: 1445.2963 - class_loss: 0.1722 - bb_acc: 0.5759 - class_acc: 0.9515    
Epoch 2/8
3777/3777 [==============================] - 42s - loss: 0.8848 - bb_loss: 847.6158 - class_loss: 0.0372 - bb_acc: 0.5830 - class_acc: 0.9931    
Epoch 3/8
3777/3777 [==============================] - 42s - loss: 0.7021 - bb_loss: 633.7780 - class_loss: 0.0684 - bb_acc: 0.7072 - class_acc: 0.9796    
Epoch 4/8
3777/3777 [==============================] - 42s - loss: 0.4029 - bb_loss: 391.3285 - class_loss: 0.0115 - bb_acc: 0.7842 - class_acc: 0.9987    
Epoch 5/8
3777/3777 [===============

In [24]:
preds_ens = []
for i in range(200):
    model.load_weights(path+'models/resnet_bb_'+str(i)+'.h5')
    preds_ens.append(model.predict(conv_test_feat, batch_size = batch_size*4)[1])
    if i % 10 == 9:
        print('Finish predicting {} samples'.format(i+1))

Finish predicting 110 samples
Finish predicting 120 samples
Finish predicting 130 samples
Finish predicting 140 samples
Finish predicting 150 samples
Finish predicting 160 samples
Finish predicting 170 samples
Finish predicting 180 samples
Finish predicting 190 samples
Finish predicting 200 samples


In [29]:
preds2_ens = []
for i in range(100, 200):
    model.load_weights(path+'models/resnet_bb_'+str(i)+'.h5')
    preds2_ens.append(model.predict(conv_test2_feat, batch_size = batch_size*4)[1])
    if i % 10 == 9:
        print('Finish predicting {} samples'.format(i+1))

Finish predicting 110 samples
Finish predicting 120 samples
Finish predicting 130 samples
Finish predicting 140 samples
Finish predicting 150 samples
Finish predicting 160 samples
Finish predicting 170 samples
Finish predicting 180 samples
Finish predicting 190 samples
Finish predicting 200 samples


In [ ]:
preds_ens_resnet_bb = np.stack(preds_ens)
preds_ens_resnet_bb.shape

In [ ]:
save_array(path+'results/preds_ens_resnet_bb.dat', preds_ens_resnet_bb)

In [30]:
preds2_ens_resnet_bb = np.stack(preds2_ens)
preds2_ens_resnet_bb.shape

(100, 12153, 8)

In [31]:
save_array(path+'results/preds2_ens_resnet_bb.dat', preds2_ens_resnet_bb)

## InceptionV3 model

### Calculate convolutional features based on pre-trained model

In [76]:
# The InceptionV3 Imagenet model
class InceptionV3():

    def __init__(self, size=(224,224), include_top=True):
        path = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.2/'
        self.FILE_PATH = path + 'inception_v3_weights_th_dim_ordering_th_kernels.h5'
        self.FILE_PATH_NO_TOP = path + 'inception_v3_weights_th_dim_ordering_th_kernels_notop.h5'
        self.create(size, include_top)

    def inception_preprocess(self, x):
        x /= 255.
        x -= 0.5
        x *= 2.
        return x

    def create(self, size, include_top):
        input_shape = (3,)+size
        img_input = Input(shape=input_shape)
        channel_axis = 1

        x = Lambda(self.inception_preprocess)(img_input)
        x = conv2d_bn(img_input, 32, 3, 3, subsample=(2, 2), border_mode='valid')
        x = conv2d_bn(x, 32, 3, 3, border_mode='valid')
        x = conv2d_bn(x, 64, 3, 3)
        x = MaxPooling2D((3, 3), strides=(2, 2))(x)

        x = conv2d_bn(x, 80, 1, 1, border_mode='valid')
        x = conv2d_bn(x, 192, 3, 3, border_mode='valid')
        x = MaxPooling2D((3, 3), strides=(2, 2))(x)

        # mixed 0, 1, 2: 35 x 35 x 256
        for i in range(3):
            branch1x1 = conv2d_bn(x, 64, 1, 1)

            branch5x5 = conv2d_bn(x, 48, 1, 1)
            branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)

            branch3x3dbl = conv2d_bn(x, 64, 1, 1)
            branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
            branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)

            branch_pool = AveragePooling2D(
                (3, 3), strides=(1, 1), border_mode='same')(x)
            branch_pool = conv2d_bn(branch_pool, 32, 1, 1)
            x = merge([branch1x1, branch5x5, branch3x3dbl, branch_pool],
                      mode='concat', concat_axis=channel_axis,
                      name='mixed' + str(i))

        # mixed 3: 17 x 17 x 768
        branch3x3 = conv2d_bn(x, 384, 3, 3, subsample=(2, 2), border_mode='valid')

        branch3x3dbl = conv2d_bn(x, 64, 1, 1)
        branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
        branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3,
                                 subsample=(2, 2), border_mode='valid')

        branch_pool = MaxPooling2D((3, 3), strides=(2, 2))(x)
        x = merge([branch3x3, branch3x3dbl, branch_pool],
                  mode='concat', concat_axis=channel_axis,
                  name='mixed3')

        # mixed 4: 17 x 17 x 768
        branch1x1 = conv2d_bn(x, 192, 1, 1)

        branch7x7 = conv2d_bn(x, 128, 1, 1)
        branch7x7 = conv2d_bn(branch7x7, 128, 1, 7)
        branch7x7 = conv2d_bn(branch7x7, 192, 7, 1)

        branch7x7dbl = conv2d_bn(x, 128, 1, 1)
        branch7x7dbl = conv2d_bn(branch7x7dbl, 128, 7, 1)
        branch7x7dbl = conv2d_bn(branch7x7dbl, 128, 1, 7)
        branch7x7dbl = conv2d_bn(branch7x7dbl, 128, 7, 1)
        branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 1, 7)

        branch_pool = AveragePooling2D((3, 3), strides=(1, 1), border_mode='same')(x)
        branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
        x = merge([branch1x1, branch7x7, branch7x7dbl, branch_pool],
                  mode='concat', concat_axis=channel_axis,
                  name='mixed4')

        # mixed 5, 6: 17 x 17 x 768
        for i in range(2):
            branch1x1 = conv2d_bn(x, 192, 1, 1)

            branch7x7 = conv2d_bn(x, 160, 1, 1)
            branch7x7 = conv2d_bn(branch7x7, 160, 1, 7)
            branch7x7 = conv2d_bn(branch7x7, 192, 7, 1)

            branch7x7dbl = conv2d_bn(x, 160, 1, 1)
            branch7x7dbl = conv2d_bn(branch7x7dbl, 160, 7, 1)
            branch7x7dbl = conv2d_bn(branch7x7dbl, 160, 1, 7)
            branch7x7dbl = conv2d_bn(branch7x7dbl, 160, 7, 1)
            branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 1, 7)

            branch_pool = AveragePooling2D(
                (3, 3), strides=(1, 1), border_mode='same')(x)
            branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
            x = merge([branch1x1, branch7x7, branch7x7dbl, branch_pool],
                      mode='concat', concat_axis=channel_axis,
                      name='mixed' + str(5 + i))

        # mixed 7: 17 x 17 x 768
        branch1x1 = conv2d_bn(x, 192, 1, 1)

        branch7x7 = conv2d_bn(x, 192, 1, 1)
        branch7x7 = conv2d_bn(branch7x7, 192, 1, 7)
        branch7x7 = conv2d_bn(branch7x7, 192, 7, 1)

        branch7x7dbl = conv2d_bn(x, 160, 1, 1)
        branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 7, 1)
        branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 1, 7)
        branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 7, 1)
        branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 1, 7)

        branch_pool = AveragePooling2D((3, 3), strides=(1, 1), border_mode='same')(x)
        branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
        x = merge([branch1x1, branch7x7, branch7x7dbl, branch_pool],
                  mode='concat', concat_axis=channel_axis,
                  name='mixed7')

        # mixed 8: 8 x 8 x 1280
        branch3x3 = conv2d_bn(x, 192, 1, 1)
        branch3x3 = conv2d_bn(branch3x3, 320, 3, 3,
                              subsample=(2, 2), border_mode='valid')

        branch7x7x3 = conv2d_bn(x, 192, 1, 1)
        branch7x7x3 = conv2d_bn(branch7x7x3, 192, 1, 7)
        branch7x7x3 = conv2d_bn(branch7x7x3, 192, 7, 1)
        branch7x7x3 = conv2d_bn(branch7x7x3, 192, 3, 3,
                                subsample=(2, 2), border_mode='valid')

        branch_pool = AveragePooling2D((3, 3), strides=(2, 2))(x)
        x = merge([branch3x3, branch7x7x3, branch_pool],
                  mode='concat', concat_axis=channel_axis,
                  name='mixed8')

        # mixed 9: 8 x 8 x 2048
        for i in range(2):
            branch1x1 = conv2d_bn(x, 320, 1, 1)

            branch3x3 = conv2d_bn(x, 384, 1, 1)
            branch3x3_1 = conv2d_bn(branch3x3, 384, 1, 3)
            branch3x3_2 = conv2d_bn(branch3x3, 384, 3, 1)
            branch3x3 = merge([branch3x3_1, branch3x3_2],
                              mode='concat', concat_axis=channel_axis,
                              name='mixed9_' + str(i))

            branch3x3dbl = conv2d_bn(x, 448, 1, 1)
            branch3x3dbl = conv2d_bn(branch3x3dbl, 384, 3, 3)
            branch3x3dbl_1 = conv2d_bn(branch3x3dbl, 384, 1, 3)
            branch3x3dbl_2 = conv2d_bn(branch3x3dbl, 384, 3, 1)
            branch3x3dbl = merge([branch3x3dbl_1, branch3x3dbl_2],
                                 mode='concat', concat_axis=channel_axis)

            branch_pool = AveragePooling2D(
                (3, 3), strides=(1, 1), border_mode='same')(x)
            branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
            x = merge([branch1x1, branch3x3, branch3x3dbl, branch_pool],
                      mode='concat', concat_axis=channel_axis,
                      name='mixed' + str(9 + i))

        if include_top:
            x = AveragePooling2D((8, 8), strides=(8, 8), name='avg_pool')(x)
            x = Flatten(name='flatten')(x)
            x = Dense(1000, activation='softmax', name='predictions')(x)
            weights_path = get_file('inception_v3_weights_th_dim_ordering_th_kernels.h5',
                                    self.FILE_PATH,
                                    cache_subdir='models',
                                    md5_hash='b3baf3070cc4bf476d43a2ea61b0ca5f')
        else:
            weights_path = get_file('inception_v3_weights_th_dim_ordering_th_kernels_notop.h5',
                                    self.FILE_PATH_NO_TOP,
                                    cache_subdir='models',
                                    md5_hash='79aaa90ab4372b4593ba3df64e142f05')

        self.img_input = img_input
        self.model = Model(self.img_input, x)
        convert_all_kernels_in_model(self.model)
        self.model.load_weights(weights_path)

In [77]:
inception = InceptionV3(size = (360, 640), include_top = False).model

In [78]:
inception.compile(optimizer = Adam(), loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [79]:
inception.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_3 (InputLayer)             (None, 3, 360, 640)   0                                            
____________________________________________________________________________________________________
convolution2d_31 (Convolution2D) (None, 32, 179, 319)  896         input_3[0][0]                    
____________________________________________________________________________________________________
batchnormalization_17 (BatchNorma(None, 32, 179, 319)  64          convolution2d_31[0][0]           
____________________________________________________________________________________________________
convolution2d_32 (Convolution2D) (None, 32, 177, 317)  9248        batchnormalization_17[0][0]      
___________________________________________________________________________________________

In [41]:
conv_trn_feat = inception.predict_generator(trn_batches, trn_batches.nb_sample)

In [39]:
conv_val_feat = inception.predict_generator(val_batches, val_batches.nb_sample)

In [42]:
conv_test_feat = inception.predict_generator(test_batches, test_batches.nb_sample)

In [46]:
save_array(path+'results/conv_trn_feat_inception.dat', conv_trn_feat)

In [47]:
save_array(path+'results/conv_val_feat_inception.dat', conv_val_feat)

In [48]:
save_array(path+'results/conv_test_feat_inception.dat', conv_test_feat)

In [135]:
conv_trn_feat = load_array(path+'results/conv_trn_feat_inception.dat')

In [136]:
conv_val_feat = load_array(path+'results/conv_val_feat_inception.dat')

In [137]:
conv_test_feat = load_array(path+'results/conv_test_feat_inception.dat')

In [138]:
conv_trn_feat.shape, conv_val_feat.shape, conv_test_feat.shape

((3277, 2048, 9, 18), (500, 2048, 9, 18), (1000, 2048, 9, 18))

### Train on fully-convolutional network

In [139]:
inception.layers[-1].output_shape[1:]

(2048, 9, 18)

In [82]:
model = Sequential([
        BatchNormalization(axis = 1, input_shape = inception.layers[-1].output_shape[1:]),
        Convolution2D(512, 3, 3, activation = 'relu', border_mode = 'same'),
        BatchNormalization(axis = 1),
        MaxPooling2D(),
        Convolution2D(256, 3, 3, activation = 'relu', border_mode = 'same'),
        BatchNormalization(axis = 1),
        MaxPooling2D(),
        Convolution2D(128, 3, 3, activation = 'relu', border_mode = 'same'),
        BatchNormalization(axis = 1),
        MaxPooling2D((1, 2)),
        Convolution2D(8, 3, 3, border_mode = 'same'),
        GlobalAveragePooling2D(),
        Activation('softmax')
    ])

th


In [83]:
model.compile(optimizer = Adam(lr=1e-3), loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [84]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
batchnormalization_310 (BatchNorm(None, 2048, 9, 18)   4096        batchnormalization_input_5[0][0] 
____________________________________________________________________________________________________
convolution2d_310 (Convolution2D)(None, 512, 9, 18)    9437696     batchnormalization_310[0][0]     
____________________________________________________________________________________________________
batchnormalization_311 (BatchNorm(None, 512, 9, 18)    1024        convolution2d_310[0][0]          
____________________________________________________________________________________________________
maxpooling2d_23 (MaxPooling2D)   (None, 512, 4, 9)     0           batchnormalization_311[0][0]     
___________________________________________________________________________________________

In [85]:
batch_size = 64

In [86]:
model.fit(conv_trn_feat, trn_labels, batch_size = batch_size, nb_epoch = 2, 
          validation_data = (conv_val_feat, val_labels))

Train on 3277 samples, validate on 500 samples
Epoch 1/2
3277/3277 [==============================] - 30s - loss: 0.8272 - acc: 0.7244 - val_loss: 2.2544 - val_acc: 0.6140
Epoch 2/2
3277/3277 [==============================] - 30s - loss: 0.2213 - acc: 0.9417 - val_loss: 0.7197 - val_acc: 0.7840


In [87]:
model.optimizer.lr = 1e-5

In [88]:
model.fit(conv_trn_feat, trn_labels, batch_size = batch_size, nb_epoch = 6, 
          validation_data = (conv_val_feat, val_labels))

Train on 3277 samples, validate on 500 samples
Epoch 1/6
3277/3277 [==============================] - 30s - loss: 0.1003 - acc: 0.9738 - val_loss: 0.4999 - val_acc: 0.8400
Epoch 2/6
3277/3277 [==============================] - 30s - loss: 0.0542 - acc: 0.9878 - val_loss: 0.2453 - val_acc: 0.9300
Epoch 3/6
3277/3277 [==============================] - 30s - loss: 0.0184 - acc: 0.9963 - val_loss: 0.1549 - val_acc: 0.9640
Epoch 4/6
3277/3277 [==============================] - 30s - loss: 0.0101 - acc: 0.9982 - val_loss: 0.1481 - val_acc: 0.9640
Epoch 5/6
3277/3277 [==============================] - 30s - loss: 0.0074 - acc: 0.9988 - val_loss: 0.1351 - val_acc: 0.9720
Epoch 6/6
3277/3277 [==============================] - 30s - loss: 0.0124 - acc: 0.9973 - val_loss: 0.1412 - val_acc: 0.9740


### Train on fully-convolutional network with bounding-box

In [147]:
inp = Input(inception.layers[-1].output_shape[1:])
x = BatchNormalization(axis = 1)(inp)
x = Convolution2D(512, 3, 3, activation = 'relu', border_mode = 'same')(x)
x = BatchNormalization(axis = 1)(x)
x = MaxPooling2D()(x)
x = Convolution2D(256, 3, 3, activation = 'relu', border_mode = 'same')(x)
x = BatchNormalization(axis = 1)(x)
x = MaxPooling2D()(x)
x = Convolution2D(128, 3, 3, activation = 'relu', border_mode = 'same')(x)
x = BatchNormalization(axis = 1)(x)
x = MaxPooling2D((1, 2))(x)
x_bb = Convolution2D(4, 3, 3, border_mode = 'same')(x)
x_bb = GlobalAveragePooling2D(name = 'bb')(x_bb)
x_class = Convolution2D(8, 3, 3, border_mode = 'same')(x)
x_class = GlobalAveragePooling2D()(x_class)
x_class = Activation('softmax', name = 'class')(x_class)

th
th


In [148]:
model = Model([inp], [x_bb, x_class])

In [149]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_5 (InputLayer)             (None, 2048, 9, 18)   0                                            
____________________________________________________________________________________________________
batchnormalization_103 (BatchNorm(None, 2048, 9, 18)   4096        input_5[0][0]                    
____________________________________________________________________________________________________
convolution2d_105 (Convolution2D)(None, 512, 9, 18)    9437696     batchnormalization_103[0][0]     
____________________________________________________________________________________________________
batchnormalization_104 (BatchNorm(None, 512, 9, 18)    1024        convolution2d_105[0][0]          
___________________________________________________________________________________________

In [150]:
model.compile(optimizer = Adam(lr=1e-3), loss = ['mse', 'categorical_crossentropy'], metrics = ['accuracy'],
             loss_weights = [1e-3, 1.])

In [151]:
model.fit(conv_trn_feat, [trn_bbox, trn_labels], batch_size = batch_size, nb_epoch = 3, 
          validation_data = (conv_val_feat, [val_bbox, val_labels]))

Train on 3277 samples, validate on 500 samples
Epoch 1/3
3277/3277 [==============================] - 31s - loss: 8.7452 - bb_loss: 7841.7386 - class_loss: 0.9034 - bb_acc: 0.6817 - class_acc: 0.7095 - val_loss: 7.4273 - val_bb_loss: 5655.3034 - val_class_loss: 1.7720 - val_bb_acc: 0.7220 - val_class_acc: 0.5620
Epoch 2/3
3277/3277 [==============================] - 31s - loss: 5.0075 - bb_loss: 4716.4867 - class_loss: 0.2910 - bb_acc: 0.7196 - class_acc: 0.9277 - val_loss: 5.6032 - val_bb_loss: 5054.8844 - val_class_loss: 0.5484 - val_bb_acc: 0.6560 - val_class_acc: 0.8680
Epoch 3/3
3277/3277 [==============================] - 31s - loss: 2.6264 - bb_loss: 2539.6282 - class_loss: 0.0868 - bb_acc: 0.5957 - class_acc: 0.9887 - val_loss: 4.9335 - val_bb_loss: 4544.5550 - val_class_loss: 0.3890 - val_bb_acc: 0.6560 - val_class_acc: 0.9000


In [152]:
model.optimizer.lr = 1e-5

In [154]:
model.fit(conv_trn_feat, [trn_bbox, trn_labels], batch_size = batch_size, nb_epoch = 8, 
          validation_data = (conv_val_feat, [val_bbox, val_labels]))

Train on 3277 samples, validate on 500 samples
Epoch 1/8
3277/3277 [==============================] - 31s - loss: 0.2328 - bb_loss: 217.8117 - class_loss: 0.0150 - bb_acc: 0.8227 - class_acc: 0.9963 - val_loss: 2.0345 - val_bb_loss: 1674.9770 - val_class_loss: 0.3595 - val_bb_acc: 0.8620 - val_class_acc: 0.9380
Epoch 2/8
3277/3277 [==============================] - 31s - loss: 0.1710 - bb_loss: 165.1694 - class_loss: 0.0058 - bb_acc: 0.8294 - class_acc: 0.9985 - val_loss: 0.7985 - val_bb_loss: 548.2933 - val_class_loss: 0.2502 - val_bb_acc: 0.8560 - val_class_acc: 0.9480
Epoch 3/8
3277/3277 [==============================] - 31s - loss: 0.1349 - bb_loss: 132.0174 - class_loss: 0.0029 - bb_acc: 0.8435 - class_acc: 0.9997 - val_loss: 0.9510 - val_bb_loss: 715.6379 - val_class_loss: 0.2353 - val_bb_acc: 0.8800 - val_class_acc: 0.9640
Epoch 4/8
3277/3277 [==============================] - 31s - loss: 0.1069 - bb_loss: 105.2789 - class_loss: 0.0017 - bb_acc: 0.8511 - class_acc: 0.9997 - val

### Conduct model ensemble based on entire training set

In [62]:
conv_ens_feat = np.concatenate((conv_trn_feat, conv_val_feat), axis = 0)

In [63]:
ens_labels = np.concatenate((trn_labels, val_labels), axis = 0)

In [89]:
conv_ens_feat.shape, ens_labels.shape

((3777, 2048, 9, 18), (3777, 8))

In [92]:
def fully_conv(i):
    model = Sequential([
        BatchNormalization(axis = 1, input_shape = inception.layers[-1].output_shape[1:]),
        Convolution2D(512, 3, 3, activation = 'relu', border_mode = 'same'),
        BatchNormalization(axis = 1),
        MaxPooling2D(),
        Convolution2D(256, 3, 3, activation = 'relu', border_mode = 'same'),
        BatchNormalization(axis = 1),
        MaxPooling2D(),
        Convolution2D(128, 3, 3, activation = 'relu', border_mode = 'same'),
        BatchNormalization(axis = 1),
        MaxPooling2D((1, 2)),
        Convolution2D(8, 3, 3, border_mode = 'same'),
        GlobalAveragePooling2D(),
        Activation('softmax')
    ])
    model.compile(optimizer = Adam(lr=1e-3), loss = 'categorical_crossentropy', metrics = ['accuracy'])
    model.fit(conv_ens_feat, ens_labels, batch_size = batch_size, nb_epoch = 2)
    model.optimizer.lr = 1e-5
    model.fit(conv_ens_feat, ens_labels, batch_size = batch_size, nb_epoch = 5)
    model.save_weights(path+'models/inception_'+str(i)+'.h5')

In [93]:
for i in range(100):
    print('Training model {}'.format(i))
    fully_conv(i)

Training model 0
th
Epoch 1/2
3777/3777 [==============================] - 33s - loss: 0.8302 - acc: 0.7212    
Epoch 2/2
3777/3777 [==============================] - 33s - loss: 0.2427 - acc: 0.9367    
Epoch 1/5
3777/3777 [==============================] - 33s - loss: 0.1641 - acc: 0.9521    
Epoch 2/5
3777/3777 [==============================] - 33s - loss: 0.0546 - acc: 0.9862    
Epoch 3/5
3777/3777 [==============================] - 33s - loss: 0.0278 - acc: 0.9955    
Epoch 4/5
3777/3777 [==============================] - 33s - loss: 0.1988 - acc: 0.9449    
Epoch 5/5
3777/3777 [==============================] - 33s - loss: 0.0637 - acc: 0.9833    
Training model 1
th
Epoch 1/2
3777/3777 [==============================] - 33s - loss: 0.8014 - acc: 0.7458    
Epoch 2/2
3777/3777 [==============================] - 33s - loss: 0.2120 - acc: 0.9391    
Epoch 1/5
3777/3777 [==============================] - 33s - loss: 0.0995 - acc: 0.9748    
Epoch 2/5
3777/3777 [===================

In [94]:
preds_ens = []
for i in range(100):
    model.load_weights(path+'models/inception_'+str(i)+'.h5')
    preds_ens.append(model.predict(conv_test_feat, batch_size = batch_size))
    if i % 10 == 9:
        print('Finish predicting {} samples'.format(i+1))

Finish predicting 10 samples
Finish predicting 20 samples
Finish predicting 30 samples
Finish predicting 40 samples


In [95]:
preds_ens_inception = np.stack(preds_ens)
preds_ens_inception.shape

(40, 1000, 8)

In [96]:
save_array(path+'results/preds_ens_inception.dat', preds_ens_inception)

## Submission

In [80]:
preds_ens_vgg = load_array(path+'results/preds_ens_vgg.dat')
preds_ens_vgg.shape

(100, 1000, 8)

In [81]:
preds_ens_resnet_v1 = load_array(path+'results/preds_ens_resnet_v1.dat')
preds_ens_resnet_v1.shape

(100, 1000, 8)

In [82]:
preds_ens_resnet_v2 = load_array(path+'results/preds_ens_resnet_v2.dat')
preds_ens_resnet_v2.shape

(100, 1000, 8)

In [83]:
preds_ens_resnet_bb = load_array(path+'results/preds_ens_resnet_bb.dat')
preds_ens_resnet_bb.shape

(100, 1000, 8)

In [84]:
#Here we split preds_ens_resnet_bb of dimension (200, 1000, 8) into two parts of equal sizes
preds_ens_resnet_bb_2 = load_array(path+'results/preds_ens_resnet_bb_2.dat')
preds_ens_resnet_bb_2.shape

(100, 1000, 8)

In [85]:
preds2_ens_vgg = load_array(path+'results/preds2_ens_vgg.dat')
preds2_ens_vgg.shape

(100, 12153, 8)

In [86]:
preds2_ens_resnet_v1 = load_array(path+'results/preds2_ens_resnet_v1.dat')
preds2_ens_resnet_v1.shape

(100, 12153, 8)

In [87]:
preds2_ens_resnet_v2 = load_array(path+'results/preds2_ens_resnet_v2.dat')
preds2_ens_resnet_v2.shape

(100, 12153, 8)

In [88]:
preds2_ens_resnet_bb = load_array(path+'results/preds2_ens_resnet_bb.dat')
preds2_ens_resnet_bb.shape

(100, 12153, 8)

In [89]:
#Here we split preds2_ens_resnet_bb of dimension (200, 12153, 8) into two parts of equal sizes
preds2_ens_resnet_bb_2 = load_array(path+'results/preds2_ens_resnet_bb_2.dat')
preds2_ens_resnet_bb_2.shape

(100, 12153, 8)

In [90]:
preds_ens = np.concatenate((preds_ens_vgg, preds_ens_resnet_v1, preds_ens_resnet_v2, 
                            preds_ens_resnet_bb, preds_ens_resnet_bb_2), axis = 0)
preds_ens.shape

(700, 1000, 8)

In [91]:
preds = preds_ens.mean(axis = 0)

In [92]:
preds2_ens = np.concatenate((preds2_ens_vgg, preds2_ens_resnet_v1, preds2_ens_resnet_v2, 
                             preds2_ens_resnet_bb, preds2_ens_resnet_bb_2), axis = 0)
preds2_ens.shape

(700, 12153, 8)

In [93]:
preds2 = preds2_ens.mean(axis = 0)

In [94]:
#use np.clip to avoid over-confident predictions
submission = pd.DataFrame(np.clip(preds, 0.005, 1-0.005), 
                          columns = ['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT'])

In [95]:
submission.insert(0, 'image', [f.split('/')[-1] for f in test_filenames])

In [96]:
submission = submission.sort_values(by = 'image')

In [97]:
submission.head()

,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
550,img_00005.jpg,0.005000,0.005000,0.005000,0.005000,0.995000,0.005000,0.005000,0.005000
748,img_00007.jpg,0.401732,0.152770,0.010288,0.016284,0.005000,0.050149,0.005000,0.366132
920,img_00009.jpg,0.989379,0.006259,0.005000,0.005000,0.005000,0.005000,0.005000,0.005000
212,img_00018.jpg,0.865520,0.078684,0.005000,0.008885,0.005000,0.026606,0.005000,0.013844
908,img_00027.jpg,0.323903,0.052104,0.005000,0.005000,0.033267,0.119276,0.027178,0.438860


In [98]:
submission2 = pd.DataFrame(np.clip(preds2, 0.005, 1-0.005), 
                          columns = ['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT'])

In [99]:
submission2.insert(0, 'image', ['test_stg2/' + f.split('/')[-1] for f in test2_filenames])

In [100]:
submission2 = submission2.sort_values(by = 'image')

In [101]:
submission2.head()

,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
5145,test_stg2/image_00001.jpg,0.512668,0.180666,0.005000,0.006530,0.168870,0.008757,0.013768,0.105565
11191,test_stg2/image_00002.jpg,0.424265,0.030756,0.005000,0.005000,0.497180,0.014084,0.006754,0.021767
3769,test_stg2/image_00003.jpg,0.553744,0.012554,0.005000,0.006163,0.320273,0.014777,0.005068,0.083118
2124,test_stg2/image_00004.jpg,0.486216,0.158614,0.073859,0.013138,0.024330,0.029429,0.039303,0.175110
948,test_stg2/image_00005.jpg,0.339345,0.014498,0.014402,0.005000,0.401382,0.049770,0.022358,0.156371


In [102]:
submission_final = pd.concat([submission, submission2])

In [103]:
submission_final.shape

(13153, 9)

In [115]:
submission_name = '../data/fish/results/submission.gz'

In [116]:
submission_final.to_csv(submission_name, index = False, compression = 'gzip')

In [117]:
FileLink(submission_name)

/home/ubuntu/courses/deeplearning1/data/fish/results/submission.gz